In [1]:
# !pip install comet-ml

In [2]:
# !pip install shap

# Imports

In [3]:
# Import before anyone else

from comet_ml import Experiment

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report
from sklearn.calibration import calibration_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report




import shap


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pickle

from collections import Counter

KeyboardInterrupt: 

In [ ]:
import sys
# sys.path.append('../ift6758/data/')
sys.path.append('../ift6758/visualizations/')

from question_5_plots import *

In [ ]:
question_no = 5

# Load Data

In [ ]:
df = pd.read_csv(r'final_df.csv', index_col = None)
print(df.columns)


#### Features from Part 4

In [ ]:
shot_type_feat = [x for x in df.columns if "shot_type_" in x]
len(shot_type_feat)

In [ ]:
new_feat = ["game_seconds", "period", 'x_coordinates','y_coordinates', 'shot_type', 'distance_from_net', 'angle_from_net',
               "previous_event_type", 'previous_event_x_coordinates', 'previous_event_y_coordinates',
              'time_since_last_event', 'distance_from_last_event', 'rebound', 'change_in_angle', 'speed',
              'time_since_powerplay_started', '5v5','4v4','3v3','5v4','5v3','4v3','4v5','3v5','3v4']
new_feat.extend(shot_type_feat)

In [ ]:
new_feat

# Data Split

In [ ]:
df_train = df[df['season'] != 20192020]
df_train = df_train[df_train['season_type'] == 'R']
df_test = df[df['season'] == 20192020]

## 5.1 Train-Valid

In [ ]:
X_1 = df_train[['angle_from_net', 'distance_from_net']]
y_1 = df_train['goal_ind']
#y = y.astype(int)
# (#BLOG)
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(X_1,y_1,test_size=0.15,random_state=10, stratify=y_1, shuffle = True)

## 5.2 Train-Valid

In [ ]:
obj_features = df_train[new_feat].select_dtypes('object').columns
obj_features

In [ ]:
def preprocess(df):
    df.drop(columns = ['shot_type'], inplace = True)
    #convert string values to numerical values
    le = LabelEncoder()
    df["previous_event_type"] = le.fit_transform(df["previous_event_type"])
    
    return df

In [ ]:
X_2 = df_train[new_feat]
y_2 = df_train['goal_ind']

X_2 = preprocess(X_2)

print(X_2.columns)
#y = y.astype(int)
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(X_2,y_2,test_size=0.15,random_state=10, stratify=y_2, shuffle = True)

# XGBoost Classifier

## 5.1 Baseline XGBoost Model

Train/Validation Setup:
We choose to 'stratify' over the target class variable, due to the high imbalance in it.
It is always desirable to split the dataset into train and validation sets in a way that preserves the same proportions of datapoints in each class as in the original complete dataset. This is important in order for the model to see fair number of examples from both the classes(0,1 in this case) during training.
Shuffle is used to prevent data from having all similar samples together, which can harm generalization capacity later.

### Create Experiment

In [ ]:
X_train_2.shape

In [ ]:
experiment_1 = Experiment(api_key = os.environ.get('COMET_API_KEY'), project_name = "milestone-2", workspace="kleitoun")

In [ ]:
experiment_1 = Experiment(api_key = os.environ.get('COMET_API_KEY'), project_name = "milestone-2", workspace="kleitoun")

### Baseline Model Training

In [ ]:
model_1 = XGBClassifier()
 
# fit the model with the training data
model_1.fit(X_train_1,y_train_1)

In [ ]:
preds_1 = model_1.predict(X_valid_1)

### Visualize Feature Importance with SHAP

In [ ]:
plot_X_train_1 = X_train_1.rename(columns = {"angle_from_net": "Shot Angle", "distance_from_net": "Shot Distance"})

In [ ]:
explainer = shap.Explainer(model_1)
shap_values = explainer(plot_X_train_1)

In [ ]:
shap.summary_plot(shap_values, X_train_1)

### Evaluation - XGBoost Baseline (5.1)

In [ ]:
# load model file, if pretrained

# model_1 = pickle.load(open("XGBoost_Baseline_model.pickle", 'rb'))


In [ ]:
#### USAGE ####
model_name_1 = '_XGBoost_Baseline'

perf_eval = Performance_Eval(model_1,model_name_1, X_train_1, y_train_1, X_valid_1, y_valid_1, question_no = question_no)
roc = perf_eval.get_roc_auc_plot()
grp = perf_eval.get_goal_rate_plot()
crp = perf_eval.get_cum_rate_plot()
cp = perf_eval.get_calibration_plot()

In [ ]:
# experiment_1.log_image("Q5_ROC.Curve.png")
# experiment_1.log_image("Q5_Goal_Rate.png")
# experiment_1.log_image("Q5_Cum_Goal.png")
# experiment_1.log_image("Q5_Calibration_Curve.png")

## Log Metrics, Model and Plots

In [ ]:
experiment_1.log_figure(figure = roc, figure_name=f'Q{question_no}_{model_name_1}_ROC_curve.png')
experiment_1.log_figure(figure = roc, figure_name=f'Q{question_no}_{model_name_1}_Goal_Rate.png')
experiment_1.log_figure(figure = roc, figure_name=f'Q{question_no}_{model_name_1}_Cum_Goal.png')
experiment_1.log_figure(figure = roc, figure_name=f'Q{question_no}_{model_name_1}_Calibration_Curve.png')


In [ ]:
y_pred_proba_1 = model_1.predict_proba(X_valid_1)[:,1]
roc_auc =  roc_auc_score(y_valid_1,y_pred_proba_1)
y_pred_1 = model_1.predict(X_valid_1)
accuracy =  accuracy_score(y_valid_1, y_pred_1)
report = classification_report(y_valid_1, y_pred_1, output_dict=True)

print(roc_auc, accuracy)
report
#result: 0.7163373991488216 0.9062212030945288

In [ ]:
metrics = {
    "roc_auc": roc_auc,
    "accuracy": accuracy,
    "classification report": report
}

In [ ]:
experiment_1.log_metrics(metrics)

In [ ]:
pickle.dump(model_1, open("XGBoost_Baseline_model.pickle", 'wb'))
t = pickle.load(open("XGBoost_Baseline_model.pickle", 'rb'))

experiment_1.log_model(name = "XGBoost_Baseline_model", file_or_folder = "XGBoost_Baseline_model.pickle")

In [ ]:
experiment_1.end

In [ ]:
experiment_1.url
#'https://www.comet.ml/kleitoun/milestone-2/b2e8d79ab6fe40cb8dfef23d74c070b8'

## 5.2

### Create Experiment

In [ ]:
experiment_2 = Experiment(api_key = os.environ.get('COMET_API_KEY'), 
                        project_name = "milestone-2", 
                        workspace="kleitoun")

### Remove NaNs and Infs

In [ ]:
def impute_df(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    return df

In [ ]:
X_train_2 = impute_df(X_train_2)

### Train a vanilla XGBoost Classifier

In [ ]:
########## WITH ONLY Q4 features
model_2 = XGBClassifier()
 
# fit the model with the training data
model_2.fit(X_train_2,y_train_2)

In [ ]:
preds_2 = model_2.predict(X_valid_2)

In [ ]:
y_pred_proba_2 = model_2.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_2)
y_pred_2 = model_2.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_2)
report = classification_report(y_valid_2, y_pred_2, output_dict=True)

print(roc_auc, accuracy)

# result: 0.7653420509819099 0.9046782247176564

# Hyper-paramter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold

In [ ]:
# A parameter grid for XGBoost
params = {
    'n_estimators': [150, 300,500],
    'learning_rate': [0.3, 0.2, 0.1],
    'max_depth': list(range(16,30,2))
}

In [ ]:
# %%time
# folds = 3
# param_comb = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
# # random search is said to be faster and more efficient at times
# random_search = RandomizedSearchCV(model_2, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_2,y_train_2), verbose=3, random_state=1001 )
# # random_search = GridSearchCV(model_2, params, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_2,y_train_2), verbose=3)


# random_search.fit(X_train_2,y_train_2)

# hp_params = random_search.best_params_


In [ ]:
hp_params = {'n_estimators': 150,
    'learning_rate': 0.3,
    'max_depth': 18}
model_2_hmtuned = XGBClassifier(**hp_params)
model_2_hmtuned.fit(X_train_2, y_train_2)

# model_2_hmtuned = model_2_hmtuned.best_estimator_


In [ ]:
y_pred_proba_2 = model_2_hmtuned.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_2)
y_pred_2 = model_2_hmtuned.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_2)
report = classification_report(y_valid_2, y_pred_2, output_dict=True)

print(roc_auc, accuracy)

# result: 0.7265493652260179 0.9025994899599254

In [ ]:
# experiment_2.log_parameters(hp_params)

## Visualize Hyperparameter Tuning Importance

#### Visualizing a few combinations, for example

In [ ]:
param_list = [{'n_estimators': 150,
    'learning_rate': 0.3,
    'max_depth': 18, 'gamma': 2},
              {'n_estimators': 300,
    'learning_rate': 0.1,
    'max_depth': 30, 'gamma': 2},
              {'n_estimators': 150,
    'learning_rate': 0.3,
    'max_depth': 18}, 
             {'n_estimators': 300,
    'learning_rate': 0.1,
    'max_depth': 30, 'gamma': 2, 'min_child_weight': 0.25}]

In [ ]:
# names = ['parameter set 1', 'parameter set 2', 'parameter set 3']
classifiers = [XGBClassifier(**param_list[0]),
               XGBClassifier(**param_list[1]),
               XGBClassifier(**param_list[2]),
               XGBClassifier(**param_list[3])
               ]

m1 = classifiers[0].fit(X_train_2, y_train_2)
m2 = classifiers[1].fit(X_train_2, y_train_2)
m3 = classifiers[2].fit(X_train_2, y_train_2)
m4 = classifiers[3].fit(X_train_2, y_train_2)

In [ ]:
model_name_2 = "2_xgboost_set_1"
perf_eval = Performance_Eval(m1, model_name_2, X_train_2, y_train_2, X_valid_2, y_valid_2,question_no = question_no)
roc_set_1 = perf_eval.get_roc_auc_plot()

In [ ]:
model_name_3 = "2_xgboost_set_2"
perf_eval = Performance_Eval(m2, model_name_3, X_train_2, y_train_2, X_valid_2, y_valid_2,question_no = question_no)
roc_set_2 = perf_eval.get_roc_auc_plot()

In [ ]:
model_name_4 = "2_xgboost_set_3"
perf_eval = Performance_Eval(m3,model_name_4, X_train_2, y_train_2, X_valid_2, y_valid_2,question_no = question_no)
roc_set_3 = perf_eval.get_roc_auc_plot()

In [ ]:
model_name_5 = "2_xgboost_set_4"
perf_eval = Performance_Eval(m4,model_name_5, X_train_2, y_train_2, X_valid_2, y_valid_2,question_no = question_no)
roc_set_4 =  perf_eval.get_roc_auc_plot()

In [ ]:
step = "hmtuning"
experiment_2.log_figure(figure = roc_set_1, figure_name=f'Q{question_no}_{step}_ROC_curve_set_1.png')
experiment_2.log_figure(figure = roc_set_2, figure_name=f'Q{question_no}_{step}_ROC_curve_set_2.png')
experiment_2.log_figure(figure = roc_set_3, figure_name=f'Q{question_no}_{step}_ROC_curve_set_3.png')
experiment_2.log_figure(figure = roc_set_4, figure_name=f'Q{question_no}_{step}_ROC_curve_set_4.png')

#### Result: param_list[3] turns out the most effective

# To avoid heavy class imbalance problem, tuning class weights scale

### Class weights

In [ ]:
from sklearn.utils import class_weight

# assigining class weight to balanced does NOT yield any good results/makes NO difference (#BLOG)

classes_weights = class_weight.compute_sample_weight(
    # based on ratio of train set class distribution
    class_weight={0:0.10, 1:0.90},
    y=y_train_2
)

In [ ]:
model_2_cw = XGBClassifier(**param_list[3])

model_2_cw.fit(X_train_2, y_train_2, sample_weight=classes_weights)

In [ ]:
y_pred_cw = model_2_cw.predict(X_valid_2)


### Evaluation - XGBoost post Class Weights (5.2)

In [ ]:
y_pred_proba_cw = model_2_cw.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_cw)
y_pred_cw = model_2_cw.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_cw)
report = classification_report(y_valid_2, y_pred_cw, output_dict=True)

print(roc_auc, accuracy)
report

#result: 0.7605648908916087 0.7943124102608062

In [ ]:
metrics = {
    "roc_auc": roc_auc,
    "accuracy": accuracy,
    "classification report": report
}

In [ ]:
experiment_2.log_metrics(metrics)

In [ ]:
#### USAGE ####
model_name_6 = '2_xgboost_hmtuning'
perf_eval = Performance_Eval(model_2_cw, model_name_6, X_train_2, y_train_2, X_valid_2, y_valid_2, question_no = question_no)
roc = perf_eval.get_roc_auc_plot()
grp = perf_eval.get_goal_rate_plot()
crp = perf_eval.get_cum_rate_plot()
cr = perf_eval.get_calibration_plot()

### Log Model and Plots

In [ ]:
experiment_2.log_figure(figure = roc, figure_name=f'Q{question_no}_{model_name_6}_ROC_curve.png')
experiment_2.log_figure(figure = grp, figure_name=f'Q{question_no}_{model_name_6}_Goal_Rate.png')
experiment_2.log_figure(figure = crp, figure_name=f'Q{question_no}_{model_name_6}_Cum_Goal.png')
experiment_2.log_figure(figure = cr, figure_name=f'Q{question_no}_{model_name_6}_Calibration_Curve.png')


In [ ]:
pickle.dump(model_2_cw, open("XGBoost_hmtuning_model_v2.pickle", 'wb'))
t = pickle.load(open("XGBoost_hmtuning_model_v2.pickle", 'rb'))

experiment_2.log_model(name = "XGBoost_hmtuning_model_v2", file_or_folder = "XGBoost_hmtuning_model_v2.pickle")

In [ ]:
experiment_2.end

In [ ]:
experiment_2.url

# Feature Selection (5.3)

### Create Experiment

In [ ]:
experiment_3 = Experiment(api_key = os.environ.get('COMET_API_KEY'), 
                        project_name = "milestone-2", 
                        workspace="kleitoun")

### Visualize Feature Importance

In [ ]:
feature_important = model_2_cw.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

viz = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
plt = viz.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features
plt.figure.savefig("Q5_feature_importance.png")

In [ ]:
experiment_3.log_image("Q5_feature_importance.png")

In [ ]:
## To also visualize the importance of class weight and stratifying the data, 
# the difference between the feature importance graphs could be helpful(#BLOG)
# explainer_2 = shap.Explainer(model_2_cw)
# shap_values_2 = explainer_2(X_train_2)

# shap.summary_plot(shap_values_2, X_train_2)

### Applying Recursive Feature Elimination

In [ ]:
#adapted from https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/
from tqdm import tqdm
from sklearn.feature_selection import SelectFromModel
#
thresholds = np.sort(model_2_cw.feature_importances_)
for thresh in tqdm(thresholds):
    # select features using threshold
    selection = SelectFromModel(model_2_cw, threshold=thresh, prefit=True)
    select_X_train_2 = selection.transform(X_train_2)
    # train model
    selection_model = XGBClassifier(**hp_params)
    selection_model.fit(select_X_train_2, y_train_2, sample_weight=classes_weights)
    # eval model
    select_X_valid_2 = selection.transform(X_valid_2)
    y_pred = selection_model.predict(select_X_valid_2)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_valid_2, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train_2.shape[1], accuracy*100.0))

### Retrain based on the optimal accuracy value

In [ ]:
## second best thresh value is prioritzed here, as the best value allows only one feature which can make the model biased
thresh = 0.020
selection = SelectFromModel(model_2_cw, threshold=thresh, prefit=True)
select_X_train_2 = selection.transform(X_train_2)

selection_model = XGBClassifier(**param_list[3])
selection_model.fit(select_X_train_2, y_train_2, sample_weight=classes_weights)


In [ ]:
cols = selection.get_support(indices=True)
transformed_X_train_2 = X_train_2.iloc[:,cols]
transformed_X_valid_2 = X_valid_2.iloc[:,cols]

In [ ]:
#### USAGE ####
model_name_7 = "3_xgboost_fs"
perf_eval = Performance_Eval(selection_model,model_name_7, transformed_X_train_2, y_train_2, transformed_X_valid_2, y_valid_2, question_no)
roc = perf_eval.get_roc_auc_plot()
grp= perf_eval.get_goal_rate_plot()
crp = perf_eval.get_cum_rate_plot()
cr = perf_eval.get_calibration_plot()

### Log Model and Plots

In [ ]:

experiment_3.log_figure(figure = roc, figure_name=f'Q{question_no}_{model_name_7}_ROC_curve.png')
experiment_3.log_figure(figure = grp, figure_name=f'Q{question_no}_{model_name_7}_Goal_Rate.png')
experiment_3.log_figure(figure = crp, figure_name=f'Q{question_no}_{model_name_7}_Cum_Goal.png')
experiment_3.log_figure(figure = cr, figure_name=f'Q{question_no}_{model_name_7}_Calibration_Curve.png')

In [ ]:
pickle.dump(selection_model, open("XGBoost_feature_selection_model_v2.pickle", 'wb'))
t = pickle.load(open("XGBoost_feature_selection_model_v2.pickle", 'rb'))

experiment_3.log_model(name = "XGBoost_feature_selection_model_v2", file_or_folder = "XGBoost_feature_selection_model_v2.pickle")

In [ ]:
experiment_3.url

In [ ]:
y_pred_proba_cw = selection_model.predict_proba(transformed_X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_cw)
y_pred_cw = selection_model.predict(transformed_X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_cw)
report = classification_report(y_valid_2, y_pred_cw, output_dict=True)

print(roc_auc, accuracy)
report

#result: 0.7593543260298987 0.7885262413475345

In [ ]:
metrics = {
    "roc_auc": roc_auc,
    "accuracy": accuracy,
    "classification report": report
}

In [ ]:
experiment_3.log_metrics(metrics)

# Experiment 4 for improvement

# SMOTE

## Does not give much of an improvement

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
# define dataset

In [ ]:
X_2 = impute_df(X_2)

In [ ]:
model = XGBClassifier(n_estimators= 150, max_depth= 4, learning_rate= 0.2, subsample= 1.0, min_child_weight= 5, gamma= 5, colsample_bytree= 1.0)
over = SMOTE(sampling_strategy="auto")
under = RandomUnderSampler(sampling_strategy="auto")
steps = [('over', over), ('under', under), ('model', model)]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X_train_2, y_train_2, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % np.mean(scores))

In [ ]:
from imblearn.over_sampling import ADASYN
from collections import Counter
# summarize class distribution
counter = Counter(y_2)
print(counter)
# transform the dataset
oversample = ADASYN()
X_train_smote, y_train_smote = oversample.fit_resample(X_2, y_2)
# summarize the new class distribution
counter = Counter(y_train_smote)
print(counter)